In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chess.pgn
import numpy as np
from IPython.display import display, clear_output
import torch
from fastai2.data import *
from fastai2.basics import *
from fastai2.callback.all import *

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
def get_bitboard(board):
    '''
    params
    ------
    board : chess.pgn board object
        board to get state from
    returns
    -------
    bitboard representation of the state of the game
    64 * 6 + 5 dim binary numpy vector
    64 squares, 6 pieces, '1' indicates the piece is at a square
    5 extra dimensions for castling rights queenside/kingside and whose turn
    '''
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard

In [5]:
def get_result(game):
    result = game.headers['Result']
    result = result.split('-')
    if result[0] == '1':
        return 1
    elif result[0] == '0':
        return -1
    else:
        return 0

In [69]:
# Change function to write files with correct names 
def pgn_to_np(path):
    games = open(path)
    bitboards = []
    labels = []
    num_games = 0

#     while True:
    for i in range(16):
#         if num_games % 1000 == 0:
#             clear_output()
#             print(num_games)
        num_games += 1

        game = chess.pgn.read_game(games)

        try:
            result = get_result(game)
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                bitboard = get_bitboard(board)

                bitboards.append(bitboard)
                labels.append(result)
        except:
            clear_output()
            print(f"{num_games} games in file")
            break

    return torch.FloatTensor(bitboards), torch.FloatTensor(labels)

bitboards, labels = pgn_to_np('data/games.pgn')

In [7]:
class ChessPosition(Transform):
    def __init__(self, bitboards, labels):
        self.bitboards, self.labels = bitboards, labels
        
    def encodes(self, i):
        return (bitboards[i], labels[i].unsqueeze(-1))

In [62]:
cp = ChessPosition(bitboards, labels)
pipe = Pipeline([cp])
splits = RandomSplitter()(range_of(range_of(pipe(0)[0])))
tls = TfmdLists(range_of(pipe(0)[0]), pipe, splits=splits)
dls = tls.dataloaders(bs=8, device=device.type, num_workers=0)

In [64]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(773, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self, feature_vec):
        hidden1 = self.linear1(feature_vec).clamp(min=0)
        output = self.linear2(hidden1)
        return F.tanh(output)

In [65]:
learn = Learner(
    dls, 
    SimpleModel(), 
    loss_func=nn.BCEWithLogitsLoss(), 
    opt_func=partial(Adam), 
    metrics=accuracy
)

In [66]:
learn.fit(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.142003,0.070747,0.155844,00:01
1,-0.060096,-0.016537,0.155844,00:01
2,-0.121328,-0.050234,0.155844,00:01
3,-0.130375,-0.049468,0.155844,00:01
4,-0.142731,-0.057911,0.155844,00:01
5,-0.158457,-0.060602,0.155844,00:01
6,-0.153500,-0.063086,0.155844,00:01
7,-0.174080,-0.064671,0.155844,00:01
8,-0.167289,-0.065925,0.155844,00:01
9,-0.160771,-0.051708,0.155844,00:01
